In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
yeast8_metabolites_path = '../../../Data/yeast-GEM-final.csv'
cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'

rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
###output
rxndb_total_info_to_model_del_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_del.csv'

rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_new.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}_new.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}_new.pickle'
metabolites_info_to_GEM_del_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}_del.csv'



In [3]:
rxndb_df = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=5,num_process=num_process)


100%|██████████| 100/100 [00:00<00:00, 292.00it/s]

number of metabolites in RXNDB: 33939


In [5]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)


100%|██████████| 33939/33939 [33:42<00:00, 16.78it/s]  


In [6]:
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)

In [8]:
sink0 = [smiles2inchikey0(x) for x in yeast8_total_smiles]

In [7]:
def process_reactions_in_parallel_reactant(rxndb,origin_smile_max_score_pd,sink0_orign, num_processes=5, num_iterations=1):
    num = 0
    tmp_smile_max_score_pd = origin_smile_max_score_pd
    while num < num_iterations:
        num+=1
        sink0 = list(set(sink0_orign))

        # pool = multiprocessing.Pool(num_processes)
        smiles_success = []
        success_rxndbid = []
        chunks = np.array_split(rxndb, 5)
        with multiprocessing.Pool(num_processes) as pool:
            for result in tqdm(pool.imap(partial(process_chunk_reactant,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                sink0_ = sink0 + [smiles2inchikey0(smile) for smile in result[0]]
                sink0_ = list(set(sink0_))
                if len(sink0_) > len(sink0):
                    sink0 = sink0_
                    success_rxndbid.extend(result[1])
                    smiles_success.extend(result[0])

            for result in tqdm(pool.imap(partial(process_chunk_product,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                sink0_ = sink0 + [smiles2inchikey0(smile) for smile in result[0]]
                sink0_ = list(set(sink0_))
                if len(sink0_) > len(sink0):
                    sink0 = sink0_
                    success_rxndbid.extend(result[1])
                    smiles_success.extend(result[0])
                # smiles_success.extend(result[0])
                # success_rxndbid.extend(result[1])
        # pool.close()
        # pool.join()
        
        smiles_success = list(set(smiles_success))
        success_rxndbid = list(set(success_rxndbid))
        # Process cumulative successful SMILES
        for smile in smiles_success:
            if get_score_from_smiles(smile, tmp_smile_max_score_pd) < 1:
                new_row = {'smile': smile, 'score': 1, 'sim_smile': 'sys'}
                tmp_smile_max_score_pd = tmp_smile_max_score_pd._append(new_row, ignore_index=True)
                # tmp_smile_max_score_pd = tmp_smile_max_score_pd.append(new_row, ignore_index=True)

        # smiles_success = [normalize_smiles(met) for met in smiles_success]
        # smiles_success = list(set(smiles_success))

        print(f'Iteration {num} - Current success count: {len(smiles_success)}')
        print(f'Iteration {num} - Current success_rxndbid count: {len(success_rxndbid)}')
        print('============================================================================')

    print('final success', len(smiles_success))
    print('final success_rxndbid', len(success_rxndbid))
    # print(success_rxndbid)
    return smiles_success, success_rxndbid,tmp_smile_max_score_pd

In [9]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd,sink0, num_processes=num_process, num_iterations=3)


100%|██████████| 100/100 [00:32<00:00,  3.11it/s]


Iteration 1 - Current success count: 10657
Iteration 1 - Current success_rxndbid count: 17840


100%|██████████| 100/100 [00:39<00:00,  2.55it/s]


Iteration 2 - Current success count: 18022
Iteration 2 - Current success_rxndbid count: 34740


100%|██████████| 100/100 [01:23<00:00,  1.19it/s]


Iteration 3 - Current success count: 28713
Iteration 3 - Current success_rxndbid count: 51340
final success 28713
final success_rxndbid 51340


In [10]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)


success number: 12320
fail number: 1990


In [11]:
def save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path):
    rxndb_to_model = rxndb_df[rxndb_df['NO'].isin(success_rxndbid)].reset_index(drop=True)
    print(rxndb_to_model.shape)
    rxndb_to_model = rxndb_to_model.drop_duplicates(subset=['templateID', 'rxn_smiles_final'], keep='first').reset_index(drop=True)
    print(rxndb_to_model.shape)
    # rxndb_to_model = rxndb_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final']]
    # print(rxndb_to_model.shape)
    rxndb_to_model.to_csv(rxndb_to_model_path,index=None)
    print(rxndb_to_model.shape)

In [12]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_total_info_to_model_del_path)


(51340, 8)
(51340, 8)
(51340, 8)


In [13]:

rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_del_path)
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
met_id_lst = []
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
for i in range(len(rxndb_to_model)):
    met_id_lst += list(rxndb_to_model['equation_dict'].iloc[i].keys())

met_id_lst = list(set(met_id_lst))
print(metabolites_info_to_GEM.shape)
metabolites_info_to_GEM_del = metabolites_info_to_GEM[metabolites_info_to_GEM['ID'].isin(met_id_lst)]
print(metabolites_info_to_GEM_del.shape)
metabolites_info_to_GEM_del.to_csv(metabolites_info_to_GEM_del_path, index=False)


(41760, 4)
(37751, 4)
